# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-06-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-06-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-07-07 04:34:00,34.223334,-82.461707,135,0,0,135.0,"Abbeville, South Carolina, US",550.413830,0.000000
1,22001.0,Acadia,Louisiana,US,2020-07-07 04:34:00,30.295065,-92.414197,1042,42,0,1000.0,"Acadia, Louisiana, US",1679.426223,4.030710
2,51001.0,Accomack,Virginia,US,2020-07-07 04:34:00,37.767072,-75.632346,1044,14,0,1030.0,"Accomack, Virginia, US",3230.597846,1.340996
3,16001.0,Ada,Idaho,US,2020-07-07 04:34:00,43.452658,-116.241552,3000,23,0,2977.0,"Ada, Idaho, US",622.940403,0.766667
4,19001.0,Adair,Iowa,US,2020-07-07 04:34:00,41.330756,-94.471059,16,0,0,16.0,"Adair, Iowa, US",223.713647,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,30616,30967,31238,31517,31836,32022,32324,32672,32951,33190
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,2330,2402,2466,2535,2580,2662,2752,2819,2893,2964
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,12968,13273,13571,13907,14272,14657,15070,15500,15941,16404


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,703,721,733,746,774,807,819,826,864,898
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,53,55,58,62,65,69,72,74,76,79
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,892,897,905,912,920,928,937,946,952,959


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,10674,12604,13934,14131,15651,16041,17331,19164,19366,20103
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1346,1384,1438,1459,1516,1559,1592,1637,1657,1702
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,9202,9371,9674,9897,10040,10342,10832,11181,11492,11884


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,247,247,253,257,267,280,280,280,280,301
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,30,30,30,30,30,31,31,31,31,31


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
432,1017.0,Chambers,Alabama,US,2020-07-07 04:34:00,32.913601,-85.390727,642,27,0,615.0,"Chambers, Alabama, US",1930.594816,4.205607
869,1055.0,Etowah,Alabama,US,2020-07-07 04:34:00,34.045673,-86.040519,750,13,0,737.0,"Etowah, Alabama, US",733.367231,1.733333
302,1011.0,Bullock,Alabama,US,2020-07-07 04:34:00,32.100305,-85.712655,369,10,0,359.0,"Bullock, Alabama, US",3653.103653,2.710027


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,2936077,130285,924148
Brazil,1623284,65487,1062542
India,719664,20159,439934


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,2936077,130285,924148,1881644,2020-07-07 04:34:00,38.306631,-91.667263
Brazil,1623284,65487,1062542,495255,2020-07-07 04:34:00,-12.669522,-48.480493
India,719664,20159,439934,259571,2020-07-07 04:34:00,23.356511,82.081064
Russia,686852,10280,453570,223002,2020-07-07 04:34:00,54.546312,62.120860
Peru,305703,10772,197619,97312,2020-07-07 04:34:00,-10.637324,-75.277744


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
New York,397649,32219,0
California,271035,6441,0
Florida,206447,3778,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
New York,397649,32219,0,365430,2020-07-07 04:34:00,42.671593,-75.579694
California,271035,6441,0,264594,2020-07-07 04:34:00,37.778248,-120.728666
Florida,206447,3778,0,202669,2020-07-07 04:34:00,28.940755,-82.700744
Texas,205642,2677,0,202965,2020-07-07 04:34:00,31.639310,-98.596074
New Jersey,173611,15229,0,158382,2020-07-07 04:34:00,40.286441,-74.622450


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,217216,23201,0
California,Los Angeles,116570,3534,0
Illinois,Cook,92781,4630,0
Arizona,Maricopa,64915,881,0
Florida,Miami-Dade,48992,1051,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,217216,23201,0,194015,2020-07-07 04:34:00,40.767273,-73.971526,36061.0
California,Los Angeles,116570,3534,0,113036,2020-07-07 04:34:00,34.308284,-118.228241,6037.0
Illinois,Cook,92781,4630,0,88151,2020-07-07 04:34:00,41.841448,-87.816588,17031.0
Arizona,Maricopa,64915,881,0,64034,2020-07-07 04:34:00,33.348359,-112.491815,4013.0
Florida,Miami-Dade,48992,1051,0,47941,2020-07-07 04:34:00,25.611236,-80.551706,12086.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-04,32672,2819,15500,855,346,68,75376,27900,8443,18165,...,286412,955,9708,6750,355,3835,10,1248,1632,698
2020-07-05,32951,2893,15941,855,346,68,77815,28606,8583,18280,...,286931,956,10020,7169,355,4277,10,1265,1632,716
2020-07-06,33190,2964,16404,855,346,70,80447,28936,8755,18365,...,287290,960,10362,7411,369,4341,10,1284,1632,734


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-04,826,74,946,52,19,3,1481,477,104,705,...,44283,28,31,62,0,13,1,337,30,8
2020-07-05,864,76,952,52,19,3,1507,484,106,706,...,44305,28,34,65,0,16,1,338,30,8
2020-07-06,898,79,959,52,19,3,1582,491,106,706,...,44321,29,37,68,0,17,1,345,30,9


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-04,19164,1637,11181,800,108,23,25930,15935,7399,16607,...,1375,840,6425,2100,340,463,8,537,1348,181
2020-07-05,19366,1657,11492,800,108,23,27597,16140,7420,16615,...,1375,849,6584,2100,340,491,8,552,1348,181
2020-07-06,20103,1702,11884,800,108,23,28531,16302,7455,16647,...,1375,858,6628,2100,341,491,8,575,1348,197


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20
3147,84080001,US,USA,840,80001.0,Out of AL,Alabama,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
52,84001095,US,USA,840,1095.0,Marshall,Alabama,US,34.369760,-86.304867,...,1389,1408,1453,1495,1537,1622,1680,1702,1711,1724
51,84001093,US,USA,840,1093.0,Marion,Alabama,US,34.136974,-87.887042,...,188,189,197,200,203,206,214,219,224,233
50,84001091,US,USA,840,1091.0,Marengo,Alabama,US,32.247676,-87.787962,...,264,266,273,277,286,299,300,302,302,312
48,84001087,US,USA,840,1087.0,Macon,Alabama,US,32.387599,-85.692677,...,160,161,165,166,170,177,181,185,194,199


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-04,42862,1107,0,94567,23209,252895,33849,46717,11996,49,...,50140,193958,24542,1238,111,65109,35247,3205,31055,1606
2020-07-05,43953,1134,0,98103,23814,264681,34048,46717,12128,49,...,51316,197421,24952,1249,111,65748,35898,3262,31577,1634
2020-07-06,44878,1162,0,101455,24253,271035,34240,46976,12293,49,...,52039,205642,25469,1251,112,66102,36985,3442,32061,1675


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-04         583     846     347  187    230     368    624     332   
2020-07-05         607     864     349  190    235     369    633     354   
2020-07-06         610     894     353  194    243     369    634     376   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-04          624       85  ...      61      4       27        6   
2020-07-05          629       97  ...      62      4       27        6   
2020-07-06          642      102  ...      66      4       27        6   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-04            102   137   183          0       39      2  
2020-07-05            106   140   184          0       39      2  
2020-07-06            113   145   190          0       39      1  

[3 rows x 3254 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-04,1007,16,0,1817,286,6334,1701,4335,512,0,...,637,2620,181,56,6,1849,1354,94,796,20
2020-07-05,1007,16,0,1825,287,6373,1701,4335,512,0,...,645,2641,184,56,6,1853,1359,94,796,20
2020-07-06,1007,16,0,1829,292,6441,1691,4338,512,0,...,652,2677,190,56,6,1853,1369,95,796,20


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-04          12       9       2    1      1      10     28       5   
2020-07-05          12       9       2    1      1      10     28       5   
2020-07-06          12       9       2    1      1      10     28       5   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-04           27        7  ...       0      0        0        0   
2020-07-05           27        7  ...       0      0        0        0   
2020-07-06           27        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-04              0     0     0         19        0      0  
2020-07-05              0     0     0         19        0      0  
2020-07-06              0     0     0         19        0      0  

[3 rows x 3254 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-04,0.010766,0.024346,0.028534,0.0,0.054878,0.000000,0.035584,0.021230,0.022155,0.006371,...,0.002187,0.003151,0.033206,0.032584,0.000000,0.150270,0.0,0.006452,0.0,0.116800
2020-07-05,0.008539,0.026250,0.028452,0.0,0.000000,0.000000,0.032358,0.025305,0.016582,0.006331,...,0.001812,0.001047,0.032138,0.062074,0.000000,0.115254,0.0,0.013622,0.0,0.025788
2020-07-06,0.007253,0.024542,0.029045,0.0,0.000000,0.029412,0.033824,0.011536,0.020040,0.004650,...,0.001251,0.004184,0.034132,0.033756,0.039437,0.014964,0.0,0.015020,0.0,0.025140


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-04,0.008547,0.027778,0.009605,0.0,0.055556,0.0,0.030619,0.017058,0.000000,0.000000,...,0.001515,0.000000,0.068966,0.050847,NaN,0.181818,0.0,0.005970,0.0,0.142857
2020-07-05,0.046005,0.027027,0.006342,0.0,0.000000,0.0,0.017556,0.014675,0.019231,0.001418,...,0.000497,0.000000,0.096774,0.048387,NaN,0.230769,0.0,0.002967,0.0,0.000000
2020-07-06,0.039352,0.039474,0.007353,0.0,0.000000,0.0,0.049768,0.014463,0.000000,0.000000,...,0.000361,0.035714,0.088235,0.046154,NaN,0.062500,0.0,0.020710,0.0,0.125000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-04,0.105764,0.028266,0.032219,0.0,0.009346,0.0,0.027989,0.029127,0.010930,0.002959,...,0.0,0.003584,0.027836,0.0,0.000000,0.000000,0.0,0.001866,0.0,0.028409
2020-07-05,0.010541,0.012217,0.027815,0.0,0.000000,0.0,0.064288,0.012865,0.002838,0.000482,...,0.0,0.010714,0.024747,0.0,0.000000,0.060475,0.0,0.027933,0.0,0.000000
2020-07-06,0.038056,0.027158,0.034111,0.0,0.000000,0.0,0.033844,0.010037,0.004717,0.001926,...,0.0,0.010601,0.006683,0.0,0.002941,0.000000,0.0,0.041667,0.0,0.088398


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-04,0.023815,0.044340,NaN,0.029334,0.025948,0.009504,0.007561,0.000000,0.006123,0.0,...,0.029315,0.038325,0.028325,0.001618,0.132653,0.011119,0.013486,0.025272,0.024343,0.015171
2020-07-05,0.025454,0.024390,NaN,0.037391,0.026067,0.046604,0.005879,0.000000,0.011004,0.0,...,0.023454,0.017854,0.016706,0.008885,0.000000,0.009814,0.018470,0.017785,0.016809,0.017435
2020-07-06,0.021045,0.024691,NaN,0.034168,0.018435,0.024006,0.005639,0.005544,0.013605,0.0,...,0.014089,0.041642,0.020720,0.001601,0.009009,0.005384,0.030280,0.055181,0.015328,0.025092


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-04      0.041071  0.021739  0.005797  0.005376  0.017699  0.002725   
2020-07-05      0.041166  0.021277  0.005764  0.016043  0.021739  0.002717   
2020-07-06      0.004942  0.034722  0.011461  0.021053  0.034043  0.000000   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-07-04      0.001605  0.031056  0.019608  0.000000  ...  0.000000    0.0   
2020-07-05      0.014423  0.066265  0.008013  0.141176  ...  0.016393    0.0   
2020-07-06      0.001580  0.062147  0.020668  0.051546  ...  0.064516    0.0   

Province_State                                                              \
Admin2         Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2020-07-04          0.0      0.0   0.020000  0.014815  0.005495        NaN   
2020-07-05          0.0      0.0   0.039216  0.021898  0.005464        NaN   
2020-07-06          0.0      0.0   0.066038  0.035714  0.032609        NaN   

Province_State                  
Admin2         Washakie Weston  
2020-07-04          0.0    0.0  
2020-07-05          0.0    0.0  
2020-07-06          0.0   -0.5  

[3 rows x 3254 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-04,0.000994,0.066667,NaN,0.010567,0.017794,0.003009,0.000000,0.000000,0.0,NaN,...,0.006319,0.007692,0.000000,0.0,0.0,0.002168,0.001479,0.010753,0.0,0.0
2020-07-05,0.000000,0.000000,NaN,0.004403,0.003497,0.006157,0.000000,0.000000,0.0,NaN,...,0.012559,0.008015,0.016575,0.0,0.0,0.002163,0.003693,0.000000,0.0,0.0
2020-07-06,0.000000,0.000000,NaN,0.002192,0.017422,0.010670,-0.005879,0.000692,0.0,NaN,...,0.010853,0.013631,0.032609,0.0,0.0,0.000000,0.007358,0.010638,0.0,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga Baldwin Barbour Bibb Blount   Bullock Butler Calhoun   
2020-07-04         0.0     0.0     0.0  0.0    0.0  0.111111    0.0     0.0   
2020-07-05         0.0     0.0     0.0  0.0    0.0  0.000000    0.0     0.0   
2020-07-06         0.0     0.0     0.0  0.0    0.0  0.000000    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-04          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-07-05          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-07-06          0.0      0.0  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-04            NaN   NaN   NaN        0.0      NaN    NaN  
2020-07-05            NaN   NaN   NaN        0.0      NaN    NaN  
2020-07-06            NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3254 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-04,0.009674,0.027478,0.027785,1.132427e-08,0.052252,-0.001678,0.038068,0.022248,0.019362,0.005748,...,0.001658,0.004185,0.033775,0.036739,0.000032,0.125678,0.000014,0.013332,0.002313,0.066855
2020-07-05,0.009107,0.026864,0.028118,5.662134e-09,0.026126,-0.000839,0.035213,0.023776,0.017972,0.006039,...,0.001735,0.002616,0.032957,0.049406,0.000016,0.120466,0.000007,0.013477,0.001157,0.046322
2020-07-06,0.008180,0.025703,0.028582,2.831067e-09,0.013063,0.014286,0.034518,0.017656,0.019006,0.005345,...,0.001493,0.003400,0.033544,0.041581,0.019726,0.067715,0.000003,0.014248,0.000578,0.035731


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-04,0.016905,0.039083,0.009353,3.739899e-08,0.075248,4.168215e-25,0.031571,0.018345,0.000007,0.000153,...,0.002184,0.002636,0.063428,0.048837,NaN,0.186048,0.0,0.015870,0.019319,0.074063
2020-07-05,0.031455,0.033055,0.007848,1.869950e-08,0.037624,2.084107e-25,0.024563,0.016510,0.009619,0.000786,...,0.001341,0.001318,0.080101,0.048612,NaN,0.208409,0.0,0.009418,0.009659,0.037032
2020-07-06,0.035403,0.036264,0.007600,9.349748e-09,0.018812,1.042054e-25,0.037166,0.015486,0.004809,0.000393,...,0.000851,0.018516,0.084168,0.047383,NaN,0.135454,0.0,0.015064,0.004830,0.081016


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-04,0.086575,0.026965,0.034105,0.000162,0.035511,0.005682,0.037040,0.030968,0.013361,0.002525,...,0.000663,0.005196,0.030686,0.036155,0.001919,0.003186,7.947286e-08,0.021089,0.001841,0.029311
2020-07-05,0.048558,0.019591,0.030960,0.000081,0.017755,0.002841,0.050664,0.021916,0.008100,0.001503,...,0.000331,0.007955,0.027716,0.018078,0.000959,0.031830,3.973643e-08,0.024511,0.000921,0.014655
2020-07-06,0.043307,0.023374,0.032535,0.000041,0.008878,0.001421,0.042254,0.015977,0.006408,0.001715,...,0.000166,0.009278,0.017200,0.009039,0.001950,0.015915,1.986821e-08,0.033089,0.000460,0.051527


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-04,0.029827,0.043118,NaN,0.039054,0.027232,0.016891,0.007982,0.000814,0.010367,2.622543e-34,...,0.033220,0.040840,0.026648,0.004627,0.092431,0.010164,0.016192,0.024180,0.021650,0.018340
2020-07-05,0.027641,0.033754,NaN,0.038223,0.026650,0.031748,0.006931,0.000407,0.010685,1.311271e-34,...,0.028337,0.029347,0.021677,0.006756,0.046216,0.009989,0.017331,0.020982,0.019230,0.017887
2020-07-06,0.024343,0.029223,NaN,0.036196,0.022542,0.027877,0.006285,0.002975,0.012145,6.556357e-35,...,0.021213,0.035494,0.021198,0.004179,0.027612,0.007687,0.023805,0.038082,0.017279,0.021490


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-04      0.028846  0.058130  0.016691  0.023709  0.025852  0.004883   
2020-07-05      0.035006  0.039703  0.011227  0.019876  0.023796  0.003800   
2020-07-06      0.019974  0.037213  0.011344  0.020464  0.028919  0.001900   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-07-04      0.008286  0.060797  0.020470  0.031599  ...  0.039490   
2020-07-05      0.011355  0.063531  0.014241  0.086388  ...  0.027942   
2020-07-06      0.006467  0.062839  0.017455  0.068967  ...  0.046229   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-07-04      0.036580  0.002185  0.000814   0.038009  0.012517  0.009105   
2020-07-05      0.018290  0.001092  0.000407   0.038612  0.017207  0.007285   
2020-07-06      0.009145  0.000546  0.000204   0.052325  0.026461  0.019947   

Province_State                                     
Admin2         Unassigned      Washakie    Weston  
2020-07-04            NaN  2.723099e-11  0.015625  
2020-07-05            NaN  1.361549e-11  0.007813  
2020-07-06            NaN  6.807746e-12 -0.246094  

[3 rows x 3254 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-04,0.009927,0.051516,NaN,0.017214,0.014262,0.007103,0.000788,0.000657,0.001393,NaN,...,0.012145,0.013482,0.010534,1.387163e-07,1.804141e-17,0.008615,0.003650,0.005428,0.002575,1.697179e-06
2020-07-05,0.004964,0.025758,NaN,0.010809,0.008879,0.006630,0.000394,0.000329,0.000697,NaN,...,0.012352,0.010748,0.013554,6.935817e-08,9.020705e-18,0.005389,0.003671,0.002714,0.001288,8.485896e-07
2020-07-06,0.002482,0.012879,NaN,0.006500,0.013151,0.008650,-0.002742,0.000510,0.000348,NaN,...,0.011602,0.012190,0.023081,3.467908e-08,4.510352e-18,0.002695,0.005515,0.006676,0.000644,4.242948e-07


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga       Baldwin Barbour Bibb Blount   Bullock   
2020-07-04      0.011882  7.152068e-15  0.2500  0.0    0.0  0.054049   
2020-07-05      0.005941  3.576034e-15  0.1250  0.0    0.0  0.027025   
2020-07-06      0.002970  1.788017e-15  0.0625  0.0    0.0  0.013512   

Province_State                                              ... Wyoming  \
Admin2            Butler   Calhoun      Chambers  Cherokee  ...    Park   
2020-07-04      0.009269  0.000062  8.384478e-08  0.000044  ...     NaN   
2020-07-05      0.004635  0.000031  4.192239e-08  0.000022  ...     NaN   
2020-07-06      0.002317  0.000015  2.096120e-08  0.000011  ...     NaN   

Province_State                                                                \
Admin2         Platte Sheridan Sublette Sweetwater Teton Uinta    Unassigned   
2020-07-04        NaN      NaN      NaN        NaN   NaN   NaN  1.797020e-06   
2020-07-05        NaN      NaN      NaN        NaN   NaN   NaN  8.985099e-07   
2020-07-06        NaN      NaN      NaN        NaN   NaN   NaN  4.492549e-07   

Province_State                  
Admin2         Washakie Weston  
2020-07-04          NaN    NaN  
2020-07-05          NaN    NaN  
2020-07-06          NaN    NaN  

[3 rows x 3254 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200706,AK,1138,122615.0,NaN,19.0,NaN,NaN,NaN,3.0,...,123753,0,0,6337485c1dab1a0798672c8aa81f714d2b339df2,0,0,0,0,0,NaN
1,20200706,AL,44878,410217.0,NaN,1016.0,2914.0,NaN,843.0,NaN,...,455095,0,5,d191311d4fceb2d234266af63c6554fdc93492e4,0,0,0,0,0,NaN
2,20200706,AR,24253,320715.0,NaN,337.0,1575.0,NaN,NaN,81.0,...,344968,6,39,dad21e873f554522affd53893c3c5e66de48d8c6,0,0,0,0,0,NaN
3,20200706,AS,0,696.0,NaN,NaN,NaN,NaN,NaN,NaN,...,696,0,0,49379bde2508bc37fd469f51e01b010fb7ffd2b8,0,0,0,0,0,NaN
4,20200706,AZ,101441,516400.0,NaN,3212.0,5188.0,839.0,NaN,533.0,...,617841,1,27,00c94733d3a298727b54688254f886a9b44c86be,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200706,AK,1138.0,122615.0,NaN,19.0,NaN,NaN,NaN,3.0,...,123753,0,0,6337485c1dab1a0798672c8aa81f714d2b339df2,0,0,0,0,0,NaN
1,20200706,AL,44878.0,410217.0,NaN,1016.0,2914.0,NaN,843.0,NaN,...,455095,0,5,d191311d4fceb2d234266af63c6554fdc93492e4,0,0,0,0,0,NaN
2,20200706,AR,24253.0,320715.0,NaN,337.0,1575.0,NaN,NaN,81.0,...,344968,6,39,dad21e873f554522affd53893c3c5e66de48d8c6,0,0,0,0,0,NaN
3,20200706,AS,0.0,696.0,NaN,NaN,NaN,NaN,NaN,NaN,...,696,0,0,49379bde2508bc37fd469f51e01b010fb7ffd2b8,0,0,0,0,0,NaN
4,20200706,AZ,101441.0,516400.0,NaN,3212.0,5188.0,839.0,NaN,533.0,...,617841,1,27,00c94733d3a298727b54688254f886a9b44c86be,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-06,AK,1138,122615.0,NaN,19.0,NaN,NaN,NaN,3.0,NaN,...,123753,0,0,6337485c1dab1a0798672c8aa81f714d2b339df2,0,0,0,0,0,NaN
2020-07-06,AL,44878,410217.0,NaN,1016.0,2914.0,NaN,843.0,NaN,473.0,...,455095,0,5,d191311d4fceb2d234266af63c6554fdc93492e4,0,0,0,0,0,NaN
2020-07-06,AR,24253,320715.0,NaN,337.0,1575.0,NaN,NaN,81.0,245.0,...,344968,6,39,dad21e873f554522affd53893c3c5e66de48d8c6,0,0,0,0,0,NaN
2020-07-06,AS,0,696.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,696,0,0,49379bde2508bc37fd469f51e01b010fb7ffd2b8,0,0,0,0,0,NaN
2020-07-06,AZ,101441,516400.0,NaN,3212.0,5188.0,839.0,NaN,533.0,NaN,...,617841,1,27,00c94733d3a298727b54688254f886a9b44c86be,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-06,AK,1138.0,122615.0,NaN,19.0,NaN,NaN,NaN,3.0,NaN,...,123753,0,0,6337485c1dab1a0798672c8aa81f714d2b339df2,0,0,0,0,0,NaN
2020-07-06,AL,44878.0,410217.0,NaN,1016.0,2914.0,NaN,843.0,NaN,473.0,...,455095,0,5,d191311d4fceb2d234266af63c6554fdc93492e4,0,0,0,0,0,NaN
2020-07-06,AR,24253.0,320715.0,NaN,337.0,1575.0,NaN,NaN,81.0,245.0,...,344968,6,39,dad21e873f554522affd53893c3c5e66de48d8c6,0,0,0,0,0,NaN
2020-07-06,AS,0.0,696.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,696,0,0,49379bde2508bc37fd469f51e01b010fb7ffd2b8,0,0,0,0,0,NaN
2020-07-06,AZ,101441.0,516400.0,NaN,3212.0,5188.0,839.0,NaN,533.0,NaN,...,617841,1,27,00c94733d3a298727b54688254f886a9b44c86be,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE